In [6]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [10]:
import os
os.chdir("/content/drive/My Drive/flipkart")
# dir_content = os.listdir("test_set_64")
# print(len(dir_content))
!ls

 cluster_images.ipynb		     GeneratedDataset.pickle    test_set_64
 configuring_dataset.ipynb	     localization_model.ipynb   train_64.csv
'Copy of localization_model.ipynb'   predictions.csv	        training.csv
 deep_learning_localization.ipynb    test.csv		        training_images
 GeneratedDataset128.pickle	     testing_images


In [0]:
import numpy as np
import os
import pandas as pd
import random
import tqdm
import cv2
os.chdir("/content/drive/My Drive/flipkart")
# !ls
train_dir = os.listdir("train_set_64")
random.shuffle(train_dir)

train_sze = int(len(train_dir)*0.8);
X_train = train_dir[:train_sze]
X_test = train_dir[train_sze:]

file = open("train_64.csv","r")
train_file = []
for line in file:
  train_file.append(line)

y_train = []
for i in tqdm.tqdm(range(len(X_train))):
  for tmp in train_file:
    tmp = tmp.split(",")
    if(tmp[0] == X_train[i]):
      X_train[i] = cv2.imread("train_set_64/"+tmp[0])
      tmp[1] = int(tmp[1])
      tmp[2] = int(tmp[2])
      tmp[3] = int(tmp[3])
      tmp[4] = int(tmp[4][:-1])
      y_train.append([tmp[1],tmp[2],tmp[3],tmp[4]])
      break


100%|██████████| 19200/19200 [09:57<00:00, 32.15it/s]


In [0]:
y_test = []
for i in tqdm.tqdm(range(len(X_test))):
  for tmp in train_file:
    tmp = tmp.split(",")
    if(tmp[0] == X_test[i]):
      X_test[i] = cv2.imread("train_set_64/"+tmp[0])
      tmp[1] = int(tmp[1])
      tmp[2] = int(tmp[2])
      tmp[3] = int(tmp[3])
      tmp[4] = int(tmp[4][:-1])
      y_test.append([tmp[1],tmp[2],tmp[3],tmp[4]])
      break


100%|██████████| 4800/4800 [02:34<00:00, 31.01it/s]


In [0]:
import pickle
import os
os.chdir("/content/drive/My Drive/flipkart")

# with open('GeneratedDataset.pickle','wb') as f:
# 	pickle.dump([X_train,y_train,X_test,y_test],f)
pickle_in = open('GeneratedDataset.pickle','rb')
[X_train,y_train,X_test,y_test] = pickle.load(pickle_in)

In [0]:
import numpy as np

X_tr = np.array(X_train)
X_sum = np.sum(X_tr)
X_avg = (X_sum / X_tr.size)
X_sq = (X_train - X_avg)**2
X_sd = np.sqrt(np.sum(X_sq)/X_tr.size + 0.001)

X_train = (X_train - X_avg)/X_sd
X_test = (X_test - X_avg)/X_sd

In [0]:
def find_accuracy(y_pred,y):
  cnt = 0
  print(y_pred.shape)
  l,_ = y_pred.shape
  for i in range(l):
    xA = [0,0,0,0]
    xB = [0,0,0,0]
    xA[0] = y_pred[i][0]-y_pred[i][2]/2
    xA[1] = y_pred[i][1]-y_pred[i][3]/2
    xA[2] = y_pred[i][0]+y_pred[i][2]/2
    xA[3] = y_pred[i][1]+y_pred[i][3]/2
    
    xA = list(map((lambda x: max(0,x)), xA))
    xA = list(map((lambda x: min(64,x)), xA))

    xB[0] = y[i][0]-y[i][2]/2
    xB[1] = y[i][1]-y[i][3]/2
    xB[2] = y[i][0]+y[i][2]/2
    xB[3] = y[i][1]+y[i][3]/2
    
    xB = list(map((lambda x: max(0,x)), xB))
    xB = list(map((lambda x: min(64,x)), xB))
    
    cnt += float(bb_intersection_over_union(xA,xB))

  cnt = cnt/len(y_pred)
  return cnt

def bb_intersection_over_union(boxA, boxB):
  xA = max(boxA[0], boxB[0])
  yA = max(boxA[1], boxB[1])
  xB = min(boxA[2], boxB[2])
  yB = min(boxA[3], boxB[3])

  interArea = max(0,(xB - xA)) * max(0,(yB - yA))
  boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
  boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

  iou = interArea / float(boxAArea + boxBArea - interArea)

  return iou

def tf_iou(boxes1, boxes2):
    def run(tb1, tb2):
        x11, y11, x12, y12 = tf.split(tb1, 4, axis=1)
        x21, y21, x22, y22 = tf.split(tb2, 4, axis=1)

        xA = tf.maximum(x11, tf.transpose(x21))
        yA = tf.maximum(y11, tf.transpose(y21))
        xB = tf.minimum(x12, tf.transpose(x22))
        yB = tf.minimum(y12, tf.transpose(y22))

        interArea = tf.maximum((xB - xA + 1), 0) * tf.maximum((yB - yA + 1), 0)

        boxAArea = (x12 - x11 + 1) * (y12 - y11 + 1)
        boxBArea = (x22 - x21 + 1) * (y22 - y21 + 1)

        iou = interArea / (boxAArea + tf.transpose(boxBArea) - interArea)

        return iou


def scaled_loss(y, y_hat):
    
    x_tmp = (y[:,0])
    y_tmp = (y[:,1])
    h_tmp = (y[:,2])
    w_tmp = (y[:,3])
    x1 = x_tmp-h_tmp/2;
    x2 = x_tmp+h_tmp/2;
    y1 = y_tmp-w_tmp/2;
    y2 = y_tmp+w_tmp/2;
    
    x_hat = (y_hat[:, 0])
    y_hat = (y_hat[:, 1])
    h_hat = (y_hat[:, 2])
    w_hat = (y_hat[:, 3])
    x1_hat = tf.subtract(x_hat,tf.divide(h_hat,2));
    x2_hat = tf.add(x_hat,tf.divide(h_hat,2));
    y1_hat = tf.subtract(y_hat,tf.divide(w_hat,2));
    y2_hat = tf.add(y_hat,tf.divide(w_hat,2));
    
#     x1_hat = y_hat[:, 0]
#     x2_hat = y_hat[:, 1]
#     y1_hat = y_hat[:, 2]
#     y2_hat = y_hat[:, 3]
    ix1 = tf.maximum(x1_hat, x1)
    ix2 = tf.minimum(x2_hat, x2)
    iy1 = tf.maximum(y1_hat, y1)
    iy2 = tf.minimum(y2_hat, y2)
    tensor_type = x1.dtype
    y_hat_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), x2_hat - x1_hat),
                             tf.maximum(tf.cast(0.0, tensor_type), y2_hat - y1_hat))
    y_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), x2 - x1),
                         tf.maximum(tf.cast(0.0, tensor_type), y2 - y1))
    i_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), ix2 - ix1),
                         tf.maximum(tf.cast(0.0, tensor_type), iy2 - iy1))
    u_area = y_hat_area + y_area - i_area
    uia = (u_area - i_area) / (y_area + 1e-8)
    return tf.reduce_mean(uia)
  
def mIOU(y, y_hat):
    x1_hat = y_hat[:, 0]
    x2_hat = y_hat[:, 1]
    y1_hat = y_hat[:, 2]
    y2_hat = y_hat[:, 3]
    x1 = y[:, 0]
    x2 = y[:, 1]
    y1 = y[:, 2]
    y2 = y[:, 3]
    ix1 = tf.maximum(x1_hat, x1)
    ix2 = tf.minimum(x2_hat, x2)
    iy1 = tf.maximum(y1_hat, y1)
    iy2 = tf.minimum(y2_hat, y2)
    tensor_type = x1.dtype
    y_hat_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), x2_hat - x1_hat),
                             tf.maximum(tf.cast(0.0, tensor_type), y2_hat - y1_hat))
    y_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), x2 - x1),
                         tf.maximum(tf.cast(0.0, tensor_type), y2 - y1))
    i_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), ix2 - ix1),
                         tf.maximum(tf.cast(0.0, tensor_type), iy2 - iy1))
    u_area = y_hat_area + y_area - i_area
    iou = i_area / u_area
    return tf.reduce_mean(iou)


In [16]:
# 96.05% for 5 epochs
# 99.21% for 10 epochs with xavier initializer
import tensorflow as tf
import sys
import numpy as np

n_classes = 4
batch_size = 128

X = tf.placeholder(tf.float32,[None,64,64,3])
y = tf.placeholder(tf.float32)

is_train = tf.placeholder(tf.bool, name="is_train");

initializer = tf.contrib.layers.xavier_initializer()

layer = {'conv1_weights':tf.Variable(initializer([3,3,3,64])),
          'conv2_weights':tf.Variable(initializer([3,3,64,64])),
          'conv3_weights':tf.Variable(initializer([3,3,64,128])),
          'conv4_weights':tf.Variable(initializer([3,3,128,128])),
          'conv5_weights':tf.Variable(initializer([3,3,128,256])),
          'conv6_weights':tf.Variable(initializer([3,3,256,256])),
          'conv7_weights':tf.Variable(initializer([3,3,256,512])),
          'conv8_weights':tf.Variable(initializer([3,3,512,512])),
          'conv9_weights':tf.Variable(initializer([3,3,512,512])),
          'fc1_weights':tf.Variable(initializer([4*4*512,4096])),
          'fc2_weights':tf.Variable(initializer([4096,4096])),
          'fc3_weights':tf.Variable(initializer([4096,1024])),
         'out_weights':tf.Variable(initializer([1024,n_classes]))}
biases = {'conv1_biases':tf.Variable(initializer([64])),
          'conv2_biases':tf.Variable(initializer([64])),
          'conv3_biases':tf.Variable(initializer([128])),
          'conv4_biases':tf.Variable(initializer([128])),
          'conv5_biases':tf.Variable(initializer([256])),
          'conv6_biases':tf.Variable(initializer([256])),
          'conv7_biases':tf.Variable(initializer([512])),
          'conv8_biases':tf.Variable(initializer([512])),
          'conv9_biases':tf.Variable(initializer([512])),
          'fc1_biases':tf.Variable(initializer([4096])),
          'fc2_biases':tf.Variable(initializer([4096])),
          'fc3_biases':tf.Variable(initializer([1024])),
          'out_biases':tf.Variable(initializer([n_classes]))}

keep_prob = tf.placeholder(tf.float32)

def cnn_model(X):
  X = tf.reshape(X,[-1,64,64,3])
  
  conv1 = tf.nn.conv2d(X,layer['conv1_weights'],strides = [1,1,1,1],padding = 'SAME') + biases['conv1_biases']
#   conv1 = tf.layers.batch_normalization(conv1, training=is_train)
  conv1 = tf.nn.relu(conv1)
#   conv1 = tf.nn.max_pool(conv1,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'SAME')

  conv2 = tf.nn.conv2d(conv1,layer['conv2_weights'],strides = [1,1,1,1],padding = 'SAME') + biases['conv2_biases']
#   conv2 = tf.layers.batch_normalization(conv2, training=is_train)
  conv2 = tf.nn.relu(conv2)
  conv2 = tf.nn.max_pool(conv2,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'SAME')

  conv3 = tf.nn.conv2d(conv2,layer['conv3_weights'],strides = [1,1,1,1],padding = 'SAME') + biases['conv3_biases']
#   conv3 = tf.layers.batch_normalization(conv3, training=is_train)
  conv3 = tf.nn.relu(conv3)
#   conv3 = tf.nn.max_pool(conv3,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'SAME')
  
  conv4 = tf.nn.conv2d(conv3,layer['conv4_weights'],strides = [1,1,1,1],padding = 'SAME') + biases['conv4_biases']
#   conv4 = tf.layers.batch_normalization(conv4, training=is_train)
  conv4 = tf.nn.relu(conv4)
  conv4 = tf.nn.max_pool(conv4,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'SAME')
  
  conv5 = tf.nn.conv2d(conv4,layer['conv5_weights'],strides = [1,1,1,1],padding = 'SAME') + biases['conv5_biases']
#   conv5 = tf.layers.batch_normalization(conv5, training=is_train)
  conv5 = tf.nn.relu(conv5)
#   conv5 = tf.nn.max_pool(conv5,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'SAME')
  
  conv6 = tf.nn.conv2d(conv5,layer['conv6_weights'],strides = [1,1,1,1],padding = 'SAME') + biases['conv6_biases']
#   conv6 = tf.layers.batch_normalization(conv6, training=is_train)
  conv6 = tf.nn.relu(conv6)
  conv6 = tf.nn.max_pool(conv6,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'SAME')
  
  conv7 = tf.nn.conv2d(conv6,layer['conv7_weights'],strides = [1,1,1,1],padding = 'SAME') + biases['conv7_biases']
#   conv7 = tf.layers.batch_normalization(conv7, training=is_train)
  conv7 = tf.nn.relu(conv7)
#   conv7 = tf.nn.max_pool(conv7,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'SAME')
  
  conv8 = tf.nn.conv2d(conv7,layer['conv8_weights'],strides = [1,1,1,1],padding = 'SAME') + biases['conv8_biases']
#   conv8 = tf.layers.batch_normalization(conv8, training=is_train)
  conv8 = tf.nn.relu(conv8)
#   conv8 = tf.nn.max_pool(conv8,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'SAME')
  
  conv9 = tf.nn.conv2d(conv8,layer['conv9_weights'],strides = [1,1,1,1],padding = 'SAME') + biases['conv9_biases']
#   conv9 = tf.layers.batch_normalization(conv9, training=is_train)
  conv9 = tf.nn.relu(conv9)
  conv9 = tf.nn.max_pool(conv9,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'SAME')

#   fc = tf.nn.conv2d(conv9,layer['fc1_weights'],strides = [1,1,1,1],padding = 'SAME') + biases['fc1_biases']
#   fc = tf.nn.relu(fc)
#   fc = tf.nn.dropout(fc,keep_prob)
  
#   fc = tf.nn.conv2d(fc,layer['fc2_weights'],strides = [1,1,1,1],padding = 'SAME') + biases['fc2_biases']
#   fc = tf.nn.relu(fc)
#   fc = tf.nn.dropout(fc,keep_prob)
  
  conv10 = tf.reshape(conv9,[-1,4*4*512])
  fc = tf.matmul(conv10,layer['fc1_weights']) + biases['fc1_biases']
#   fc = tf.layers.batch_normalization(fc, training=is_train)
  fc = tf.nn.relu(fc)
  fc = tf.nn.dropout(fc,keep_prob)
  
  fc = tf.matmul(fc,layer['fc2_weights']) + biases['fc2_biases']
#   fc = tf.layers.batch_normalization(fc, training=is_train)
  fc = tf.nn.relu(fc)
  fc = tf.nn.dropout(fc,keep_prob)
  
  fc = tf.matmul(fc,layer['fc3_weights']) + biases['fc3_biases']
#   fc = tf.layers.batch_normalization(fc, training=is_train)
  fc = tf.nn.relu(fc)
  fc = tf.nn.dropout(fc,keep_prob)

  classification = tf.matmul(fc,layer['out_weights']) + biases['out_biases']
  
  return classification

y_predict = cnn_model(X)

# y_pred = np.array(y_predict)
# y_ans = np.array(y)
# cost = tf.reduce_mean(tf.reduce_sum(tf.abs(tf.subtract(y_predict, y)),axis = 1))

cost = tf.reduce_mean(tf.nn.l2_loss(tf.subtract(y_predict,y)))
# cost = scaled_loss(y_predict,y)

# update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
# with tf.control_dependencies(update_ops):
model = tf.train.AdamOptimizer(0.0001).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())


# y1 = sess.run(y_predict,feed_dict = {X:X_train[:5],y:y_train[:5],keep_prob:1.0})
# y2 = sess.run(y,feed_dict = {X:X_train[:5],y:y_train[:5],keep_prob:1.0})
# # y1 = np.array(y1)
# # y2 = np.array(y2)
# print(tf.reduce_sum(tf.math.abs(y1-y2),axis = 1))

num_iter = 50

for i in range(num_iter):
    iter_loss=0
    ind = 0
    for j in range(int(len(X_train)/128)):
        batch_x = X_train[ind:ind+128]
        batch_y = y_train[ind:ind+128]
        ind+=128
        _,c=sess.run([model,cost],feed_dict = {X:batch_x,y:batch_y,keep_prob:0.5,is_train:True})
        iter_loss += c
    print(i,iter_loss)
    

# accuracy = accuracy(y_predict,y)
# print(sess.run(accuracy,feed_dict = {X:X_test,y:y_test,keep_prob:1.0}))
# sess.close()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
0 7082869.162109375
1 3397997.3359375
2 1937950.7060546875
3 1666180.9008789062
4 1546924.6420898438
5 1413217.4560546875
6 1321551.05078125
7 1253567.2172851562
8 1221114.033203125
9 1158621.1342773438
10 1094923.38671875
11 1064427.0080566406
12 1005679.0913085938
13 935316.4487304688
14 919653.005859375
15 882098.5729980469
16 826858.1657714844
17 768681.130859375
18 740025.9560546875
19 683395.2141113281
20 662412.5615234375
21 646981.7141113281
22 649122.3696289062
23 642719.3801269531
24 729627.169921875
25 634568.4885253906
26 603494.3767089844
27 685273.757324218

In [28]:
ans_y = sess.run(y_predict,feed_dict = {X:X_test[:500],keep_prob:1})
actual_y = sess.run(y,feed_dict = {y:y_test[:500],keep_prob:1})
accuracy = find_accuracy(np.array(ans_y),np.array(actual_y))
print(accuracy)

TypeError: ignored

In [22]:
import matplotlib.pyplot as plt
import cv2
import copy
import os
import numpy as np
import tqdm

testing_dir = os.listdir("testing_images")

for img_name in testing_dir[20:40]:
  image = cv2.imread("testing_images/" + img_name)
  image2 = cv2.resize(image,(64,64))
  img2 = np.reshape(image2,(-1,64,64,3))
  c=sess.run(y_predict,feed_dict = {X:img2,keep_prob:1})
  img = image
  x = int(round(c[0][0]))
  y = int(round(c[0][1]))
  h = int(round(c[0][2]))
  w = int(round(c[0][3]))
  x1 = x-int(h/2);
  x2 = x+int(h/2);
  y1 = y-int(w/2);
  y2 = y+int(w/2);
  [x1,y1,x2,y2] = list(map((lambda x: max(0,x)),[x1,y1,x2,y2]))
  [x1,y1,x2,y2] = list(map((lambda x: min(63,x)),[x1,y1,x2,y2]))
  x1 = int(round((x1*480)/64))
  x2 = int(round((x2*480)/64))
  y1 = int(round((y1*640)/64))
  y2 = int(round((y2*640)/64))
  
  print(x1,y1,x2,y2)
  for i in range(x1,x2):
    img[i][y1][0] = 0;
    img[i][y1][0] = 128;
    img[i][y1][0] = 0;
    img[i][y1+1][0] = 0;
    img[i][y1+1][0] = 128;
    img[i][y1+1][0] = 0;
  for i in range(y1,y2):
    img[x1][i][0] = 0;
    img[x1][i][0] = 128;
    img[x1][i][0] = 0;
  for i in range(y1,y2):
    img[x2-1][i][0] = 0;
    img[x2-1][i][0] = 128;
    img[x2-1][i][0] = 0;
  for i in range(x1,x2):
    img[i][y2-1][0] = 0;
    img[i][y2-1][0] = 128;
    img[i][y2-1][0] = 0;
  image = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  plt.figure()
  plt.imshow(image)


Output hidden; open in https://colab.research.google.com to view.

 configuring_dataset.ipynb	     localization_model.ipynb   train.csv
'Copy of localization_model.ipynb'   predictions.csv	        training.csv
 deep_learning_localization.ipynb    test.csv		        training_images
 GeneratedDataset.pickle	     testing_images	        train_set
